In [1]:
import sys
sys.path.append("../")

# 📥 STEP 1: Load GPT-Generated Strategies
import json
from pathlib import Path

strategies_path = Path("data/strategies_generated.json")

if strategies_path.exists():
    with open(strategies_path, "r") as f:
        strategies = json.load(f).get("strategies", [])
    print(f"✅ Loaded {len(strategies)} strategies.")
else:
    print("❌ strategies_generated.json not found.")
    strategies = []












✅ Loaded 3 strategies.


In [2]:
# 🧠 STEP 2: Format strategies for GPT ranking
formatted_strategies = "\n\n".join([
    f"{i+1}. 📊 **{s['strategy_name']}**\n"
    f"- Description: {s['description']}\n"
    f"- Why this fits the trader: {s['why_it_fits']}\n"
    f"- Best conditions for this strategy: {s['best_conditions']}"
    for i, s in enumerate(strategies)
])

print("📦 Preview of formatted strategies:")
print(formatted_strategies[:1000])








📦 Preview of formatted strategies:
1. 📊 **Iron Condor**
- Description: An iron condor involves selling a call spread and a put spread on the same underlying asset with the same expiration date. This creates a range where the trade can profit.
- Why this fits the trader: This strategy suits traders who have a knack for managing multiple positions, can stay calm under pressure and are comfortable with limited price movements – all characteristics shown by the trader.
- Best conditions for this strategy: Ideal for a market that's expected to move sideways or within a relatively tight range.

2. 📊 **Diagonal Call Spread**
- Description: A diagonal call spread involves buying a longer-term call option and selling a shorter-term call option with a higher strike price. This strategy profits from both time decay and a moderate upward move in the underlying asset.
- Why this fits the trader: The trader has shown success with calendar spreads, which are similar in structure but benefit from diff

In [3]:
# 🤖 STEP 3: Compose prompt and call GPT
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

ranking_prompt = f"""
You are a senior options strategist.

Rank the following option strategies from best to worst based on:
1. Profit Potential
2. Fit for the trader based on their past performance
3. Risk Management Flexibility

Return only your ranked list and a short reason per strategy.

{formatted_strategies}
"""

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a trading strategist ranking expert."},
        {"role": "user", "content": ranking_prompt}
    ],
    temperature=0.5
)

ranking_output = response.choices[0].message.content.strip()
print("📄 GPT-Enhanced Ranking Preview:\n")
print(ranking_output)








📄 GPT-Enhanced Ranking Preview:

1. 📊 **Diagonal Call Spread**
   - Ranking Reason: This strategy has the most potential for profit as it benefits from both time decay and upward movement in the underlying asset's price. The trader's past performance with calendar spreads suggests they have the necessary skills to effectively manage this strategy. The risk can be managed by adjusting the strike prices and expiration dates of the options.

2. 📊 **Iron Condor**
   - Ranking Reason: Iron Condor comes second as it offers moderate profit potential within a defined range. It fits well with the trader's ability to manage multiple positions and handle pressure. The risk management flexibility is relatively high because the trader can adjust the strike prices and widths of the spreads to control the level of risk.

3. 📊 **Butterfly Spread**
   - Ranking Reason: This strategy has the least profit potential as it only profits if the underlying asset's price stays close to the middle strike price.

In [4]:
# 🧩 STEP 4: Parse the ranked output to structured format

def parse_ranked_strategies(raw_text):
    import re

    blocks = re.split(r"\n\d+\.\s", "\n" + raw_text.strip())
    parsed = []

    for block in blocks[1:]:
        lines = block.strip().split("\n")
        name_line = lines[0]
        strategy_name = name_line.strip("** ").strip()
        rationale = " ".join(lines[1:]).strip()

        parsed.append({
            "strategy_name": strategy_name,
            "rank": len(parsed) + 1,
            "reason": rationale
        })
    
    return parsed

structured_ranking = parse_ranked_strategies(ranking_output)
print(f"🧪 Parsed {len(structured_ranking)} ranked strategies")
print(structured_ranking[:2])








🧪 Parsed 3 ranked strategies
[{'strategy_name': '📊 **Diagonal Call Spread', 'rank': 1, 'reason': "- Ranking Reason: This strategy has the most potential for profit as it benefits from both time decay and upward movement in the underlying asset's price. The trader's past performance with calendar spreads suggests they have the necessary skills to effectively manage this strategy. The risk can be managed by adjusting the strike prices and expiration dates of the options."}, {'strategy_name': '📊 **Iron Condor', 'rank': 2, 'reason': "- Ranking Reason: Iron Condor comes second as it offers moderate profit potential within a defined range. It fits well with the trader's ability to manage multiple positions and handle pressure. The risk management flexibility is relatively high because the trader can adjust the strike prices and widths of the spreads to control the level of risk."}]


In [5]:
# 💾 STEP 5: Save to ranked_strategies.json
output_path = Path("data/ranked_strategies.json")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "w") as f:
    json.dump({"ranked_strategies": structured_ranking}, f, indent=2)

print(f"✅ Saved ranked strategies to {output_path}")





✅ Saved ranked strategies to data/ranked_strategies.json
